# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 13 2022 9:14AM,241734,10,0085867800,ISDIN Corporation,Released
1,Jul 13 2022 9:14AM,241734,10,0085867808,ISDIN Corporation,Released
2,Jul 13 2022 9:14AM,241734,10,0085867830,ISDIN Corporation,Released
3,Jul 13 2022 9:14AM,241734,10,0085867829,ISDIN Corporation,Released
4,Jul 13 2022 9:14AM,241734,10,0085867881,ISDIN Corporation,Released
5,Jul 13 2022 9:14AM,241734,10,0085867969,ISDIN Corporation,Released
6,Jul 13 2022 9:14AM,241734,10,0085868029,ISDIN Corporation,Released
7,Jul 13 2022 9:14AM,241734,10,0085868041,ISDIN Corporation,Released
8,Jul 13 2022 9:14AM,241734,10,0085868050,ISDIN Corporation,Released
9,Jul 13 2022 9:14AM,241734,10,0085868052,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,241731,Released,16
45,241732,Released,1
46,241733,Released,36
47,241734,Released,18
48,241735,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241731,NaN,NaN,16.0
241732,NaN,NaN,1.0
241733,NaN,NaN,36.0
241734,NaN,NaN,18.0
241735,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241604,0.0,0.0,22.0
241635,0.0,3.0,8.0
241644,0.0,0.0,1.0
241648,0.0,0.0,5.0
241654,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241604,0,0,22
241635,0,3,8
241644,0,0,1
241648,0,0,5
241654,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241604,0,0,22
1,241635,0,3,8
2,241644,0,0,1
3,241648,0,0,5
4,241654,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241604,,,22
1,241635,,3,8
2,241644,,,1
3,241648,,,5
4,241654,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 13 2022 9:14AM,241734,10,ISDIN Corporation
18,Jul 13 2022 9:12AM,241735,10,"Senseonics, Incorporated"
19,Jul 13 2022 9:10AM,241733,10,ISDIN Corporation
55,Jul 13 2022 9:10AM,241731,10,ISDIN Corporation
71,Jul 13 2022 9:09AM,241730,10,ISDIN Corporation
133,Jul 13 2022 9:02AM,241732,10,"Citieffe, Inc."
134,Jul 13 2022 8:46AM,241729,10,Eye Pharma Inc
136,Jul 13 2022 8:42AM,241727,10,ISDIN Corporation
150,Jul 13 2022 8:42AM,241728,10,"Methapharm, Inc."
156,Jul 13 2022 8:37AM,241724,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 13 2022 9:14AM,241734,10,ISDIN Corporation,,,18
1,Jul 13 2022 9:12AM,241735,10,"Senseonics, Incorporated",,,1
2,Jul 13 2022 9:10AM,241733,10,ISDIN Corporation,,,36
3,Jul 13 2022 9:10AM,241731,10,ISDIN Corporation,,,16
4,Jul 13 2022 9:09AM,241730,10,ISDIN Corporation,,,62
5,Jul 13 2022 9:02AM,241732,10,"Citieffe, Inc.",,,1
6,Jul 13 2022 8:46AM,241729,10,Eye Pharma Inc,,,2
7,Jul 13 2022 8:42AM,241727,10,ISDIN Corporation,,,14
8,Jul 13 2022 8:42AM,241728,10,"Methapharm, Inc.",,,6
9,Jul 13 2022 8:37AM,241724,10,"Methapharm, Inc.",,,11


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 9:14AM,241734,10,ISDIN Corporation,18,,
1,Jul 13 2022 9:12AM,241735,10,"Senseonics, Incorporated",1,,
2,Jul 13 2022 9:10AM,241733,10,ISDIN Corporation,36,,
3,Jul 13 2022 9:10AM,241731,10,ISDIN Corporation,16,,
4,Jul 13 2022 9:09AM,241730,10,ISDIN Corporation,62,,
5,Jul 13 2022 9:02AM,241732,10,"Citieffe, Inc.",1,,
6,Jul 13 2022 8:46AM,241729,10,Eye Pharma Inc,2,,
7,Jul 13 2022 8:42AM,241727,10,ISDIN Corporation,14,,
8,Jul 13 2022 8:42AM,241728,10,"Methapharm, Inc.",6,,
9,Jul 13 2022 8:37AM,241724,10,"Methapharm, Inc.",11,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 9:14AM,241734,10,ISDIN Corporation,18,,
1,Jul 13 2022 9:12AM,241735,10,"Senseonics, Incorporated",1,,
2,Jul 13 2022 9:10AM,241733,10,ISDIN Corporation,36,,
3,Jul 13 2022 9:10AM,241731,10,ISDIN Corporation,16,,
4,Jul 13 2022 9:09AM,241730,10,ISDIN Corporation,62,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 9:14AM,241734,10,ISDIN Corporation,18.0,NaN,NaN
1,Jul 13 2022 9:12AM,241735,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Jul 13 2022 9:10AM,241733,10,ISDIN Corporation,36.0,NaN,NaN
3,Jul 13 2022 9:10AM,241731,10,ISDIN Corporation,16.0,NaN,NaN
4,Jul 13 2022 9:09AM,241730,10,ISDIN Corporation,62.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 9:14AM,241734,10,ISDIN Corporation,18.0,0.0,0.0
1,Jul 13 2022 9:12AM,241735,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Jul 13 2022 9:10AM,241733,10,ISDIN Corporation,36.0,0.0,0.0
3,Jul 13 2022 9:10AM,241731,10,ISDIN Corporation,16.0,0.0,0.0
4,Jul 13 2022 9:09AM,241730,10,ISDIN Corporation,62.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7009320,291.0,26.0,7.0
15,483313,42.0,0.0,0.0
16,725172,23.0,0.0,0.0
18,241669,0.0,1.0,0.0
19,966800,31.0,3.0,0.0
20,725040,38.0,1.0,0.0
21,966759,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7009320,291.0,26.0,7.0
1,15,483313,42.0,0.0,0.0
2,16,725172,23.0,0.0,0.0
3,18,241669,0.0,1.0,0.0
4,19,966800,31.0,3.0,0.0
5,20,725040,38.0,1.0,0.0
6,21,966759,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,291.0,26.0,7.0
1,15,42.0,0.0,0.0
2,16,23.0,0.0,0.0
3,18,0.0,1.0,0.0
4,19,31.0,3.0,0.0
5,20,38.0,1.0,0.0
6,21,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,291.0
1,15,Released,42.0
2,16,Released,23.0
3,18,Released,0.0
4,19,Released,31.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,7.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,26.0,0.0,0.0,1.0,3.0,1.0,0.0
Released,291.0,42.0,23.0,0.0,31.0,38.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,7.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,26.0,0.0,0.0,1.0,3.0,1.0,0.0
2,Released,291.0,42.0,23.0,0.0,31.0,38.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,7.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,26.0,0.0,0.0,1.0,3.0,1.0,0.0
2,Released,291.0,42.0,23.0,0.0,31.0,38.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()